_Anmerkung: Ggf. müssen sie nach dem Öffnen des Notebooks noch einmal auf 'Kernel > Restart & Run All' clicken_

<h1>Demonstrator: Abfragen mit SPARQL</h1>

In diesem Demonstrator werden wir zeigen, wie SPARQL Abfragen genutzt werden können, 
um aus Open Data Beständen weiteres Wissen zu ziehen.

![SPARQL Logo](./media/SPARQL.png)

<h2> Übersicht </h2>

Im Verlauf dieses Vortrages werden wir nochmal die Grundlagen von RDF und SPARQL-Abfragen erklären, 
und danach darauf eingehen, wie sich derart gewonnene Daten zur Erweiterung eines bestehenden Datenbestandes Nutzen lassen. <br><br>
Idealerweise können sie am Ende dieses Demonstrators beim ihren nächsten Projekt ebenfalls Daten über Open Data im RDF-Format erweitern. <br>
In diesem Vortrag kommen daher folgende Themen vor:

    - Was ist SPARQL 
    - SPARQL Queries
    - Erweiterung einer DB mit Open Data
        - Suche in der DB
        - Geeignete Attribute zur Identifikation über mehrere DB
        - Erweiterung der DB
    - Welche Daten sind (in der erweiterten DB) nicht vorhanden
    - Fazit

<h2> Was ist SPARQL? </h2>

Bei SPARQL handelt es sich um die **S**PARQL **P**rotocol **A**nd **R**DF **Q**uery **L**anguage. <br>
Es ist also sowohl ein Protokoll als auch eine Abfragesprache für das Resource Description Framework. <br>
Während SQL die Abfragesprache für relationale Datenbanken ist, ist SPARQL die geläufige Abfragesprache für RDF-Daten, welche nicht als Tabellen sondern als Graph mit gelabelten Kanten zu verstehen sind.
RDF steht für **R**esource **D**escription **F**ramework, ein vom W3C standardisiertes, XML-basiertes Format zur Beschreibung von Datenobjekten und ihren Beziehungen zueinander. <br><br><br>


![RDF Graph mit Beziehungen zwischen Personen](./media/RDF_Graph.png)


<br><br>In diesem Graph kann man unter anderem ablesen, das Mary die Frau von John ist. <br>
Als Subjekt Prädikat Objekt Triple lässt sich dies als<br>
*John hasSpouse Mary* <br>
formulieren. Und SPARQL erlaubt, diese Abfragen in Queries über das auf HTTP basierte SPARQL-Protokoll an einen SPARQL Query Endpoint zu senden. Die Ergebnisse können in mehreren Formaten zurückgeliefert werden, die gängigen Formate sind jedoch JSON und XML. <br><br>

<h2> SPARQL Queries </h2>

Eine SPARQL Abfrage ähnelt in ihren Augen möglicherweise einer SQL-Abfrage, jedoch sind die Gemeinsamkeiten nur oberflächlich.
Abfragen besitzen einen SELECT-Part, der die resultierenden Variablen (von der Struktur den Spalten in SQL entsprechend) wiedergibt und einem WHERE-Part, der definiert, welche Eigenschaften die resultierenden Filme alle gemeinsam besitzen.

In [8]:
from data.sparql import query, table

table(query("""
SELECT ?movie
WHERE
{
 ?movie wdt:P31 wd:Q11424
}
LIMIT 10
"""))

movie
http://www.wikidata.org/entity/Q372
http://www.wikidata.org/entity/Q593
http://www.wikidata.org/entity/Q595
http://www.wikidata.org/entity/Q1365
http://www.wikidata.org/entity/Q2201
http://www.wikidata.org/entity/Q2345
http://www.wikidata.org/entity/Q2875
http://www.wikidata.org/entity/Q3092
http://www.wikidata.org/entity/Q3187
http://www.wikidata.org/entity/Q3208


Diese Query liefert die ersten 10 Filme, welche in Wikidata gelistet sind.
Zwar können sie auf die Einträge klicken, um zu sehen, um was es sich handelt, jedoch ist dieses Format noch nicht sonderlich leserlich.

Um statt der Objekte selbst ihre Label zu erhalten, erweitern wir die vorherige Query. Diesmal soll nicht der Film selbst, sondern sein Titel angezeigt werden.

In [9]:
from data.sparql import query, table

table(query(
"""
SELECT ?title
WHERE
{
 ?movie wdt:P31 wd:Q11424 .
 ?movie wdt:P1448 ?title       
} 
LIMIT 10
"""
))    

title
...men Olsenbanden var ikke død
Le guerriere dal seno nudo
La vendetta dei gladiatori
Zinebi Festival Internacional de Cine Documental y Cortometraje de Bilbao
Relatives
Börn
Syurpriz
Petualang Tak Kenal Menyerah
طلق صناعي
Brexit (2019)


Wie sie sehen, muss nicht jede Variable auch ausgegeben werden.
Hier sieht man nur den Titel, aber nicht das Film-Objekt.

Zudem ist dies die erste Query, bei der mehrere Kriterien (nämlich Objekt ?movie ist vom Typ Film und Object ?movie hat Titel ?title.
Jedes dieser logischen Tripletts wird durch einen Punkt vom nächsten getrennt.
Um etwas Schreibarbeit zu sparen, kann auch 

```
SELECT ?title
WHERE
{
 ?movie wdt:P31 wd:Q11424;
        wdt:P1448 ?title       
} 
LIMIT 10
```

schreiben. Hier wird durch das Semikolon ";" das Subjekt aus dem letzten Triplet für das Nächste wiederverwendet.

Schreiben sie ebenfalls eine Abfrage, welche die IMDB ID dreier Filme ausgibt.
Das Prädikat für 'hat IMDB ID' lautet 'wdt:P345', bei der IMDB ID handelt es sich um eine ID der 'Internet Movie Database'.

In [10]:
# Tragen sie hier ihren SPARQL-Code ein und klicken sie auf RUN, um ihn auszutesten.
# Denken sie daran, ggf. über 'from data.sparql import query, table' die notwendigen Funktionen zu importieren,
# sollte es sie hier zu einem Fehler kommen.








<h2> Erweiterung einer DB mit Open Data </h2>

Sie haben nun einen Überblick über die grundlegende Struktur von SPARQL Abfragen bekommen.<br>
Die durch SPARQL verfügbar gemachten Daten können verwendet werden, 
um beispielsweise zusätzliche Informationen zu denen einer Datenbank hinzuzufügen. <br><br>

In diesem Notebook handelt es sich hierbei um eine MongoDB, welche eine vielzahl an Filmen enthält.
Die Daten entstammen den bekannten *movie_metadata.csv* File.
Zur Rekapitulation: Dieser Datensatz enthält folgende Informationen zu Filmen bis zum Jahr 2020:

- index (Integer, n-ter Eintrag innerhalb des Datensatzes)
- title
- adult (Boolean, ob der Film FSK 18 ist)
- belongs_to_collection
- budget
- genres
- homepage
- id (Integer, ID innerhalb des Datensatz)
- imdb_id (String, ID innerhalb der Internet Movie Database)
- original_language
- original_title
- overview (String, sehr kurze Textzusammenfassung der Handlung)
- popularity
- poster_path
- production_companies
- production_countries
- release_date
- revenue
- runtime
- spoken_languages
- status (String, Released, produziert aber nicht veröffentlicht, etc.)
- tagline (String, Slogan, Untertitel des Films)
- video
- vote_average
- vote_count


Die Datenbank enthält also bereits viele Informationen.
Titel, Genre, Veröffentlichungszeitpunkt und ähnliches ist enthalten. <br>
Jedoch fehlen auch einige Informationen in den bisherigen Daten.
Beispielsweise: <br>
- Welche Schauspieler:innen haben in den Film gespielt? (Bzw. wer hat allgemein am Film mitgewirkt) 
- In welchen Ländern wurde der Film veröffentlicht?
- Welche Altersfreigabe gibt es **pro Land**?

Diese im Original-Datensatz nicht vorhandenen Informationen können gegebenenfalls über Open Data im Netz in Erfahrung gebracht werden. 
Dies geschieht im folgenden.
In der Datenbank befindet sich beispielsweise zum Film Iron Man folgendes:

In [11]:
import data.movies as movies

iron_man_movies = movies.get_by_title('Iron Man')

table(iron_man_movies)

Key,Value
index,12588
adult,False
belongs_to_collection,Iron Man Collection
budget,140000000
genres,"Action,Science Fiction,Adventure"
homepage,http://www.ironmanmovie.com/
id,1726
imdb_id,tt0371746
original_language,en
original_title,Iron Man


Um diesen Film, sowie die anderen, in der Datenbank befindlichen Einträge um zusätzliche Informationen zu erweitern, muss zunächst ein geeigneter eindeutiger Identifier für einen Film gefunden werden. <br>
Dazu eignen sich einige Einträge eines Films besser als andere. <br><br>

Frage: Welche der Eigenschaften des (oben sichtbaren) Filmes haltet ihr für sinvoll um online nach Infos zu suchen?<br>
Und warum?





<h3> Suche nach Namen: </h3>

Im folgenden suchen wir zuerst Filme nach Namen

In [12]:
from data.sparql import query, data, table, entity_to_json

#movie is instance (P31) of film (Q11424)
#movie has label ?titel

table(query(
"""
SELECT *
WHERE
{
 ?movie wdt:P31 wd:Q11424 ;
        rdfs:label ?title ;
  FILTER CONTAINS(?title, "Iron Man")
  FILTER LANGMATCHES(lang(?title),'en')
} 
"""
))


movie,title
http://www.wikidata.org/entity/Q185884,Tetsuo: The Iron Man
http://www.wikidata.org/entity/Q192724,Iron Man
http://www.wikidata.org/entity/Q192724,Iron Man
http://www.wikidata.org/entity/Q192724,Iron Man
http://www.wikidata.org/entity/Q205028,Iron Man 2
http://www.wikidata.org/entity/Q205028,Iron Man 2
http://www.wikidata.org/entity/Q205028,Iron Man 2
http://www.wikidata.org/entity/Q209538,Iron Man 3
http://www.wikidata.org/entity/Q209538,Iron Man 3
http://www.wikidata.org/entity/Q209538,Iron Man 3


Hier haben wir viele Ergebnisse, welche ggf. hilfreich sind, wenn der Nutzer im allgemeinen nach Schlagworten sucht.
Für die Exploration eines Datensatzes sinnvoll, für die Erweiterung von Daten jedoch nicht geeignet. Hier wollen wir eher ein Eindeutiges Attribut haben und der Name allein (ohne weitere Merkmale wie Veröffentlichungsjahr und Regisseur) ist dies nicht.

<h3> Suche nach imdb_id: </h3>

Die IMDB enthält viele Informationen zu Filmen und listet Filme unter der eigenen ID. <br>
Diese IMDB ID kann genutzt werden, um ggf. mehr Daten aus Open Data herauszufinden.

In [13]:
from data.sparql import query, table

table(query(
"""
SELECT *
WHERE
{
 ?movie wdt:P31 wd:Q11424 ;
        wdt:P345 ?imdb_id .
 FILTER CONTAINS (?imdb_id, "tt0371746")
} 
LIMIT 1
"""
))



movie,imdb_id
http://www.wikidata.org/entity/Q192724,tt0371746


Auf WikiData finden wir über die IMDB ID ein Datenobjekt. <br>
Als Nutzer können sie auf den Link dieses Datenobjektes klicken und sich die Wikidata-Seite ansehen.
Zur maschinellen Verarbeitung kann aber auch das Objekt in als JSON angefragt und dann weiter verarbeitet werden.

In [14]:
from data.sparql import query, data, table, entity_to_json

result = query(
"""
SELECT *
WHERE
{
 ?movie wdt:P31 wd:Q11424 ;
        rdfs:label ?title ;
        wdt:P345 ?imdb_id .
  FILTER CONTAINS(?title, "Iron Man")
  FILTER LANGMATCHES(lang(?title),'en')
} 
LIMIT 1
"""
)

# a bit hacky, but it retrieves the id from the wikidata URI, which has the form https://www.wikidata.org/wiki/{id}
url = data(result)[0]['movie']
entity_id = url.split("/").pop()

print(url)
entity_to_json(entity_id)

http://www.wikidata.org/entity/Q205028


{'entities': {'Q205028': {'pageid': 201380,
   'ns': 0,
   'title': 'Q205028',
   'lastrevid': 1432306514,
   'modified': '2021-05-31T12:35:21Z',
   'type': 'item',
   'id': 'Q205028',
   'labels': {'zh-hans': {'language': 'zh-hans', 'value': '钢铁侠2'},
    'zh-hant': {'language': 'zh-hant', 'value': '鋼鐵人2'},
    'zh-hk': {'language': 'zh-hk', 'value': '鐵甲奇俠2'},
    'zh-cn': {'language': 'zh-cn', 'value': '钢铁侠2'},
    'zh-sg': {'language': 'zh-sg', 'value': '钢铁侠2'},
    'zh-tw': {'language': 'zh-tw', 'value': '鋼鐵人2'},
    'pl': {'language': 'pl', 'value': 'Iron Man 2'},
    'he': {'language': 'he', 'value': 'איירון מן 2'},
    'fr': {'language': 'fr', 'value': 'Iron Man 2'},
    'ko': {'language': 'ko', 'value': '아이언맨 2'},
    'es': {'language': 'es', 'value': 'Iron Man 2'},
    'ms': {'language': 'ms', 'value': 'Iron Man 2'},
    'lv': {'language': 'lv', 'value': 'Dzelzs vīrs 2'},
    'it': {'language': 'it', 'value': 'Iron Man 2'},
    'de': {'language': 'de', 'value': 'Iron Man 2'},
 

Hier erkennt man, dass wir in diesem JSON Objekt eine große Menge an Daten zum Film erhalten. 
Wir können hier den Film-Titel in vielen Sprachen angeben, herausfinden, welcher Direktor für den Schnitt des Filmes verantwortlich ist, welche unterschiedlichen Jugendschutz-Label ein Film hat, etc.

<h3> Einfügen aller Schauspieler eines Films </h3>

In [15]:
from data.sparql import query, data, table, entity_to_json

result = query(
"""
SELECT *
WHERE
{
 ?movie wdt:P31 wd:Q11424 ;
        rdfs:label ?title ;
        wdt:P345 ?imdb_id .
  FILTER CONTAINS(?title, "Iron Man")
  FILTER LANGMATCHES(lang(?title),'en')
} 
LIMIT 1
"""
)

Einige der Eigenschaften sind jedoch über ihre Wikidata-ID aufgelistet. Diese muss aufgelöst werden, um menschenverständliche Daten zu erhalten, also z.B. das es sich bei 'P345' um eine 'IMDB ID' handelt. 

In [16]:
from data.sparql import query, data, table, entity_to_json

result = query(
"""
SELECT ?titleLabel ?castLabel ?property ?propLabel
WHERE {
    ?title wdt:P345 "tt0371746".
    # take all claims on this movie
    ?title ?property ?cast .
    # that involve a human
    ?cast wdt:P31 wd:Q5 .

    # get the property label
    # see https://www.wikidata.org/wiki/Wikidata:SPARQL_query_service/queries#Adding_labels_for_properties
    hint:Query hint:optimizer "None" .
    ?prop wikibase:directClaim ?property .

    SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
"""
)

table (result)

titleLabel,castLabel,property,propLabel
Iron Man,Matthew Libatique,http://www.wikidata.org/prop/direct/P344,director of photography
Iron Man,Dan Lebental,http://www.wikidata.org/prop/direct/P1040,film editor
Iron Man,Stan Lee,http://www.wikidata.org/prop/direct/P1431,executive producer
Iron Man,Jon Favreau,http://www.wikidata.org/prop/direct/P1431,executive producer
Iron Man,Peter Billingsley,http://www.wikidata.org/prop/direct/P1431,executive producer
Iron Man,David Maisel,http://www.wikidata.org/prop/direct/P1431,executive producer
Iron Man,Louis D'Esposito,http://www.wikidata.org/prop/direct/P1431,executive producer
Iron Man,Ari Arad,http://www.wikidata.org/prop/direct/P1431,executive producer
Iron Man,Laura Jean Shannon,http://www.wikidata.org/prop/direct/P2515,costume designer
Iron Man,Rebecca Bentjen,http://www.wikidata.org/prop/direct/P2515,costume designer


Die so abgerufene Liste an Schauspielern bzw. Mitwirkenden dieses Films wird nun dem Datensatz der lokalen Datenbank hinzugefügt. Dazu werden die Daten zunächst aufbereitet und dann in dem Datensatz gespeichert.

Der Datensatz enthält daraufhin die Schauspieler als ein weiteres Attribut:

In [17]:
from data.sparql import query, data, table, entity_to_json
import data.movies as movies


result = query(
"""
SELECT ?titleLabel ?castLabel ?property ?propLabel
WHERE {
    ?title wdt:P345 "tt0371746".
    # take all claims on this movie
    ?title ?property ?cast .
    # that involve a human
    ?cast wdt:P31 wd:Q5 .

    # get the property label
    # see https://www.wikidata.org/wiki/Wikidata:SPARQL_query_service/queries#Adding_labels_for_properties
    hint:Query hint:optimizer "None" .
    ?prop wikibase:directClaim ?property .

    SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
"""
)

dict = {"cast_members": [{"name": cast["castLabel"]["value"], "role": cast["propLabel"]["value"]} for cast in result["results"]["bindings"]]}
#print (dict)


movies.add_data_by_title("Iron Man", dict)

table(movies.get_by_title('Iron Man'))

Key,Value
index,12588
adult,False
belongs_to_collection,Iron Man Collection
budget,140000000
genres,"Action,Science Fiction,Adventure"
homepage,http://www.ironmanmovie.com/
id,1726
imdb_id,tt0371746
original_language,en
original_title,Iron Man


Dieser Vorgang kann nun automatisiert für alle Einträge der lokalen Datenbank durchlaufen werden. 

In [ ]:
from data.sparql import query, data, table, entity_to_json
import json
import data.movies as movies

def wikidata_query (imdb_id):
    return query(
    f"""
    SELECT ?titleLabel ?castLabel ?property ?propLabel
    WHERE {{
        ?title wdt:P345 "{imdb_id}".
        # take all claims on this movie
        ?title ?property ?cast .
        # that involve a human
        ?cast wdt:P31 wd:Q5 .

        # get the property label
        # see https://www.wikidata.org/wiki/Wikidata:SPARQL_query_service/queries#Adding_labels_for_properties
        hint:Query hint:optimizer "None" .
        ?prop wikibase:directClaim ?property .

        SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en". }}
    }}
    """
    )

def extract_cast (wikidata_result):
    dict = {"cast_members": []}
    for cast_member in result["results"]["bindings"]:
        actor = {
            "Name": cast_member["castLabel"]["value"],
            "Role": cast_member["propLabel"]["value"]
        }
        dict["cast_members"].append(actor)
    return dict


#load all movies from local db (limited to 5):
local_movies = movies.get_all(10000)
for movie in local_movies:
    imdb_id = movie["imdb_id"]
    title = movie["original_title"]
    movie_id = movie["id"]
 
    #Query an 
    result = wikidata_query(imdb_id)
    
    cast_dict = extract_cast(result)
    
    movies.add_data_by_title(title, cast_dict)
    #print(json.dumps(movies.get_by_title(title), indent=1))
    
    print(f"handled movie: '{title}' with imdb {imdb_id}")


handled movie: 'Toy Story' with imdb tt0114709
handled movie: 'Jumanji' with imdb tt0113497
handled movie: 'Grumpier Old Men' with imdb tt0113228
handled movie: 'Waiting to Exhale' with imdb tt0114885
handled movie: 'Father of the Bride Part II' with imdb tt0113041
handled movie: 'Heat' with imdb tt0113277
handled movie: 'Sabrina' with imdb tt0114319
handled movie: 'Tom and Huck' with imdb tt0112302
handled movie: 'Sudden Death' with imdb tt0114576
handled movie: 'GoldenEye' with imdb tt0113189
handled movie: 'The American President' with imdb tt0112346
handled movie: 'Dracula: Dead and Loving It' with imdb tt0112896
handled movie: 'Balto' with imdb tt0112453
handled movie: 'Nixon' with imdb tt0113987
handled movie: 'Cutthroat Island' with imdb tt0112760
handled movie: 'Casino' with imdb tt0112641
handled movie: 'Sense and Sensibility' with imdb tt0114388
handled movie: 'Four Rooms' with imdb tt0113101
handled movie: 'Ace Ventura: When Nature Calls' with imdb tt0112281
handled movie: '

handled movie: 'Casper' with imdb tt0112642
handled movie: 'Clockers' with imdb tt0112688
handled movie: 'Congo' with imdb tt0112715
handled movie: 'Crimson Tide' with imdb tt0112740
handled movie: 'Crumb' with imdb tt0109508
handled movie: 'Desperado' with imdb tt0112851
handled movie: 'Devil in a Blue Dress' with imdb tt0112857
handled movie: 'Die Hard: With a Vengeance' with imdb tt0112864
handled movie: 'The Doom Generation' with imdb tt0112887
handled movie: 'Feast of July' with imdb tt0113044
handled movie: 'First Knight' with imdb tt0113071
handled movie: 'Free Willy 2 - The Adventure Home' with imdb tt0113114
handled movie: 'Hackers' with imdb tt0113243
handled movie: 'Jeffrey' with imdb tt0113464
handled movie: 'Johnny Mnemonic' with imdb tt0113481
handled movie: 'Judge Dredd' with imdb tt0113492
handled movie: 'Jury Duty' with imdb tt0113500
handled movie: 'Kids' with imdb tt0113540
handled movie: 'Living in Oblivion' with imdb tt0113677
handled movie: 'Lord of Illusions' wit

Jetzt können wir beliebige, weitere Filme aus der Datenbank laden und den erweiterten Datenbestand nutzen

In [ ]:
import json
import data.movies as movies
from data.sparql import table

table(movies.get_by_title("Dingo"))




**Frage**: *Wie kann man hier die Performance erhöhen?* 

Eine Anwendung kann diese Zusatzdaten zum Beispiel weiter nutzen, um ggf. bei Bedarf weitere Daten zu den Schauspielern, also den einzelnen cast_members, zu laden. 

In [ ]:
from data import movies
from data.sparql import query, table

dingo = movies.get_by_title("Dingo")
cast_members = dingo['cast_members']

for member in cast_members:
    member_name = member['Name']

    q = (""
    "SELECT DISTINCT ?name ?image ?birthday ?nationalityLabel "
    "WHERE { "
    "  ?person rdfs:label \""+member_name+"\"@en; "
    "          wdt:P27 ?nationality. "
    "BIND(\""+member_name+"\" AS ?name) "
    "OPTIONAL { "
    "    ?person wdt:P18 ?image; "
    "            wdt:P569 ?birthday. "
    "} "
    "SERVICE wikibase:label { bd:serviceParam wikibase:language \"en\". } "        
    "} "
    "LIMIT 1")
    
    
    table(query(q))

<h2> Welche Daten sind nicht vorhanden </h2>

Obwohl wir nun weit mehr Daten zur Verfügung haben, könnte man sich weitere, nicht vorhandene Daten vorstellen. <br>
Wir wissen, welche Schauspieler im Film mitgespielt haben, welche Nationalität sie haben, wie alt sie sind -<br>
wir könnten sogar den Altersschnitt der Film-Crew berechnen!<br><br>

Wir wissen jedoch nicht
- Wie lang ein Film in den Kinos lief
- In welchen Kinos ein Film gelaufen ist
- Auf welchen Sendern der Film vielleicht schon im Fernsehen gelaufen ist
- Welche Kritiken es im Wortlaut zu dem Film gab
- Welche Soundtracks / Musik im Film verwendet wurde

<br><br>
Der Grund, dass wir diese Eigenschaften nicht kennen, ist, dass sie nicht im Wikidata-Datensatz vorhanden sind. 
Dies muss jedoch nicht heißen, dass wir dies überhaupt nicht herausfinden können.
Gäbe es eine Plattform, auf der Kinos ihre Spielpläne und Filmhistorien als RDF-Graph bereit stellen würden, könnten wir dies nutzen. Das selbe gilt z.B. für die deutschen Fernsehsender.
Für Musik-Informationen gibt es tatsächlich einen offenen SPARQL-Endpoint auf MusicBrainz, das heißt hier könnte man (möglicherweise) die Musik von bekannten Filmen wiederfinden. <br>


Probleme <br>
Zu welchen problemen kann es bei der Erweiterung von Daten kommen und wie kann auf diese reagiert werden?

- Es können Daten schlichtweg nicht vorhanden sein (Lösung: setze sie auf null/None oder ähnliche defaults)
- Es können Daten datenbestandsübergreifend nicht korrekt sein, beispiel: Die IMDB_ID auf Wikidata ist anders als die in der MongoDB (Lösung: Ggf. Kontrolle über weitere Eigenschaft wie Titel)

<h2> Fazit </h2>

SPARQL ermöglicht es, Linked Open Data im Netz abzufragen. <br>
Über Attribute wie gemeinsam bekannte Datenbank-ID's oder andere, gebräuchliche Identifier (ISBN, DOI, etc.) können bereits bestehende Datenbestände einfach erweitert werden. <br>
Auf diese Art und Weise kann man weitere Informationen wie Schauspieler eines Filmes ergänzen. <br>
Die Daten können durch unterschiedliche Quellen und Ontologien erweitert werden, beispielsweise können über GeoNames zu den bestehenden Daten wie Film-Ort weitere Ortsinformationen bereit gestellt werden. <br>
Würde man all diese Informationen stehts als RDF über SPARQL Queries abrufen können, dann hätte man tatsächlich ein semantisches Internet
